In [1]:
import spacy
import json
from spacy import displacy
import re
import pandas as pd
from configparser import ConfigParser
import string

/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def get_annotation(file):
    
    filename,ents= file
    #print(filename)
    data = json.load(open('./OCR_Files/'+filename, encoding="utf-8"))
    txt = ''
    annotations=[]
    for pages in data['analyzeResult']['readResults']:
        txt=''
        for line in pages['lines']:
            txt = txt+' '+line['text']
        #print(txt,'\n\n')
        para = txt
        txt = txt.lower()
        ner_tag = {}
        tags=[]
        for ent in ents:
            txt2find,tag = ent
            txt2find = txt2find.lower()
            start=0
            for i,character in enumerate(txt):
                flag=0
                temp=''
                if character == txt2find[0]:
                    start = i
                    j=i
                    k=0
                    while((k<len(txt2find))&(j<len(txt)-1)):
                        if(txt[j]==txt2find[k]):
                            temp=temp+txt[j]
                            j=j+1
                            k=k+1
                        elif(txt2find[k] in string.punctuation):
                            k=k+1
                        elif(txt[j] in string.punctuation):
                            temp=temp+txt[j]
                            j=j+1
                        else:
                            end=j
                            break
                if ((len(temp)>=len(txt2find))):
                    tags.append((start, start+len(temp),tag))
        if len(tags)!=0:
            ner_tag['entities'] = tags
            annotations.append((para,ner_tag))
    return(annotations)

In [13]:
from ast import literal_eval
annotations = []
with open('./train_both.txt', 'r', encoding="utf-8") as f:
    data = f.read().split(',\n')
for d in data:
    #print(literal_eval(d))
    annotations.append(literal_eval(d))
len(annotations)

2075

In [14]:
annotations[340]

('ADR-2021-100000726690.txt',
 [('Benicia Unified School District - 001', 'ORG'),
  ('YOLANDA M. VALDIVIA', 'PERSON')])

In [15]:
SAMPLES = []
length = []
count = 0
i=0
for annot in annotations:
    #print(i)
    i=i+1
    entities = []
    for entries in annot[1]:
        entities.append(entries[1])
    entities = list(set(entities))
    
    try:
        annotation = get_annotation(annot)
        if len(annotation)!=0:
            SAMPLES = SAMPLES + annotation
        else:
            count=count+1
            #print(annot[0])
    except FileNotFoundError:
        pass
print('Total samples ->', len(SAMPLES))
print('Total no match found ->', count)

Total samples -> 4419
Total no match found -> 24


### Spacy input format

In [17]:
SAMPLES[0]

(' 04/30/2021 Randall A. Mangali 0.00 ***NON-NEGOTIABLE*** ****This is not a check. *****Advice of deposit only**** Randall A. Mangali 373 Pantano Cir MARTINEZ CA 94553-5638 EMPLOYER PAY PERIOD ASTRON TEL Period Beginning 04/12/2021 831-38 ROUTE 10 STE 236 Period Ending: 04/25/2021 WHIPPANY NJ 07981 Pay Date: 04/30/2021 973-975-4237 Total Hours: 58.00 EMPLOYEE Randall A. Mangali 373 Pantano Cir MARTINEZ CA 94553-5638 SS#: ...3949 NET PAY: $3,041.70 Acct#....2179: $3,041.70 MEMO: PAY Hours Rate Current YTD DEDUCTIONS Current YTD HOURLy 1 18.00 42.00 756.00 18,900.00 Regular Pay 40.00 21.00 840.00 14,280.00 EXP REIMB 680.40 10,101.46 - - Nontaxable Per Diem 920.00 15,640.00 - - TAXES Current YTD Federal Income Tax 13.45 2,403.92 SUMMARY Current YTD Social Security 98.95 2,057.16 Medicare 23.14 481.11 Total Pay $3,196.40 $58,921.46 CA Income Tax Taxes $154.70 $6,269.85 0.00 929.50 CA State Disability Ins 19.16 398.16 Deductions $0.00 $0.00 Net Pay $3,041.70',
 {'entities': [(192, 202, 'OR

### TRAIN TEST SPLIT ***

In [18]:
import random
random.shuffle(SAMPLES)

In [19]:
TRAIN_DATA = SAMPLES[:10000]
VAL_DATA = SAMPLES[10000:12295]
TEST_DATA = SAMPLES[12295:12396]

In [20]:
print('no. of train samples: ', len(TRAIN_DATA))
print('no. of val samples: ', len(VAL_DATA))
print('no. of test samples: ', len(TEST_DATA))

no. of train samples:  4419
no. of val samples:  0
no. of test samples:  0


In [13]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model
nlp = spacy.load('en_core_web_trf', exclude=['tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
#nlp = spacy.load('en_core_web_trf')

def spacy_format(DATA, val_data):
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            #print(span)
            if span is None:
                #print("Skipping entity")
                pass
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    
    if val_data==False:
        db.to_disk("./train.spacy") # save the docbin object
    else:
        db.to_disk("./val.spacy")

In [14]:
spacy_format(TRAIN_DATA, val_data=False)
spacy_format(VAL_DATA, val_data=True)

100%|██████████| 2295/2295 [00:15<00:00, 150.68it/s]


In [47]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

/usr/bin/python: No module named spacy


In [ ]:
!python -m spacy train config.cfg --output /mnt/spacy/ --paths.train ./train.spacy --paths.dev ./val.spacy --gpu-id 0

In [ ]:
ls

### EVALUATION

In [96]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

def evaluate(ner_model, samples):
    scorer = Scorer(ner_model)
    examples = []
    for input_, annot in samples:
        pred = ner_model(input_)
        example = Example.from_dict(pred, annot)
        examples.append(example)
    scores = scorer.score(examples)    
    return scores

In [98]:
import numpy as np
ner = spacy.load('/mnt/spacy/model-last')

In [99]:
evaluate(ner, TEST_DATA)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.8546511627906976,
 'ents_r': 0.91875,
 'ents_f': 0.8855421686746987,
 'ents_per_type': {'ORG': {'p': 0.8070175438596491,
   'r': 0.8932038834951457,
   'f': 0.847926267281106},
  'PERSON': {'p': 0.9482758620689655,
   'r': 0.9649122807017544,
   'f': 0.9565217391304347}}}